<a href="https://colab.research.google.com/github/syyoun-25/25-2-intelligent-language-processing-exercise/blob/main/word_embeddings_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec (CBOW vs. Skip-gram), GloVe, and 2D Visualization

1) 공개 데이터셋 **text8**로 **Word2Vec** 모델을 CBOW/Skip-gram 방식으로 각각 학습
2) **사전 학습된 GloVe (glove-wiki-gigaword-100)** 모델로 단어 관계 확인 실습
3) **t-SNE**로 선택한 단어 벡터를 2D로 시각화

**Colab 사용**: 런타임 환경 연결 -- 구글 드라이브 연결 -- 의존성 설치


In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# 의존성 설치 (Colab 최초 실행 시)
!pip install gensim
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

In [2]:
import gensim.downloader as api
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

## Section 1: Word2Vec 모델 학습 (CBOW vs. Skip-gram)

In [ ]:
# ================================================================= #
# Section 1: Word2Vec 모델 학습 (CBOW vs. Skip-gram)
# ================================================================= #

print("="*60)
print("1. Word2Vec 모델 학습 (CBOW vs. Skip-gram)")
print("공개 데이터셋으로 두 학습 방식의 차이를 비교해 봅시다.")
print("="*60)

# 공개 데이터셋 'text8' 다운로드
# 'text8': 약 170만 개 단어 토큰으로 구성된 단일 텍스트 파일 -- Wikipedia 글 일부를 선형적으로 연결한 텍스트
print("\n공개 데이터셋 'text8'을 다운로드합니다...")
try:
    corpus = list(api.load('text8'))
    print("다운로드 완료!")
except Exception as e:
    print(f"\n데이터셋 다운로드 중 오류가 발생했습니다: {e}")
    print("인터넷 연결을 확인하거나 gensim-data 설치를 다시 시도해주세요.")
    raise SystemExit

# 데이터셋의 파이썬 객체 타입 확인
print("\n[데이터셋의 타입]:", type(corpus))

# 데이터셋의 첫 번째 리스트 확인 (각 리스트는 10,000개의 단어로 구성되어 있다.)
print("\n[데이터셋의 첫 번째 문장(단어 리스트) 확인]:")
print(corpus[0])

# 데이터셋의 첫 번째 문장의 타입 확인
print("\n[첫 번째 문장의 타입]:", type(corpus[0]))

# 데이터셋의 첫 번째 문장에 포함된 단어의 수 확인
print("\n[첫 번째 문장에 포함된 단어의 수]:", len(corpus[0]))

# 데이터셋의 총 리스트 수와 총 단어 수 확인
total_sentences = len(corpus)
total_words = sum(len(sentence) for sentence in corpus)
print("\n[데이터셋 통계]")
print(f"총 문장(리스트) 수: {total_sentences}")
print(f"총 단어 수: {total_words}")
print("="*60)

# CBOW (Continuous Bag of Words) 모델 학습
# sg=0: 주변 단어들을 이용해 중간 단어를 예측 (빠르고 효율적)
print("\nCBOW 모델 학습...")
cbow_model = Word2Vec(
    sentences=corpus,
    vector_size=100, # 워드 벡터의 특징 값, 즉 임베딩된 벡터의 차원
    window=5,        # 컨텍스트 윈도우 크기
    min_count=100,   # 단어 최소 빈도 수 제한 (최소 출현 빈도 100회 미만 단어는 무시)
    sg=0             # 0은 CBOW
)

# skip-gram 모델 학습
# sg=1: 중간 단어를 이용해 주변 단어들을 예측 (성능이 더 좋지만 학습 속도가 상대적으로 느림)
print("\nSkip-gram 모델 학습...")
skipgram_model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=100,
    sg=1              # 1은 skip-gram
)

print("\n[CBOW 모델 결과 - 'king'과 가장 유사한 단어 3개]")
print(cbow_model.wv.most_similar('king', topn=3))

print("\n[Skip-gram 모델 결과 - 'king'과 가장 유사한 단어 3개]")
print(skipgram_model.wv.most_similar('king', topn=3))

# 모델 저장하기
cbow_model.save("cbow_model.model")
skipgram_model.save("skipgram_model.model")

## Section 2: 사전 학습된 GloVe 모델 활용

In [ ]:
# =================================================================
# Section 2: 사전 학습된 GloVe 모델 활용
# =================================================================

print("\n\n" + "="*60)
print("2. 사전 학습된 GloVe 모델 활용")
print("="*60)

# gensim에서 제공하는 'glove-wiki-gigaword-100' (GloVe 모델)을 다운로드
print("\n사전 학습된 'glove-wiki-gigaword-100' 모델 다운로드...")
try:
    glove_model = api.load('glove-wiki-gigaword-100')
    print("다운로드 완료!")

    # 단어 관계 추론
    # 'king' - 'man' + 'woman' = ? -> 'queen'
    # 'seoul' - 'korea' + 'japan' = ? -> 'tokyo'
    print("\n[단어 관계 추론 실습]")
    result = glove_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
    print(f"king - man + woman = {result[0][0]}")

    result_city = glove_model.most_similar(positive=['seoul', 'japan'], negative=['korea'], topn=1)
    print(f"seoul - korea + japan = {result_city[0][0]}")

    result_city2 = glove_model.most_similar(positive=['france', 'tokyo'], negative=['paris'], topn=1)
    print(f"france - paris + tokyo = {result_city2[0][0]}")

except Exception as e:
    print(f"\n모델 다운로드 또는 사용 중 오류가 발생했습니다: {e}")
    print("인터넷 연결을 확인하거나 라이브러리 설치를 다시 시도해주세요.")


## Section 3: 워드 벡터 시각화 (t-SNE)

In [ ]:
# =================================================================
# Section 3: 워드 벡터 시각화
# =================================================================

print("\n\n" + "="*60)
print("3. 워드 벡터 시각화")
print("추상적인 벡터를 2D 공간에 시각화하여 단어의 관계를 이해합니다.")
print("="*60)

# 모델 로드하기
loaded_cbow_model = Word2Vec.load("cbow_model.model")
loaded_skipgram_model = Word2Vec.load("skipgram_model.model")

# 시각화할 단어 리스트를 정의합니다.
words_to_visualize = ['king', 'queen', 'man', 'woman', 'berlin', 'germany', 'paris', 'france', 'apple', 'orange', 'computer', 'science']

# (안전장치) 모델의 vocabulary에 존재하는 단어만 사용
present_words_cbow = [w for w in words_to_visualize if w in loaded_cbow_model.wv.key_to_index]
present_words_skip = [w for w in words_to_visualize if w in loaded_skipgram_model.wv.key_to_index]
if len(present_words_cbow) < len(words_to_visualize) or len(present_words_skip) < len(words_to_visualize):
    print("일부 단어가 vocabulary에 없어 제외되었습니다.")
    print("CBOW 사용 단어:", present_words_cbow)
    print("Skip-gram 사용 단어:", present_words_skip)

vectors_cbow = [loaded_cbow_model.wv[word] for word in present_words_cbow]
vectors_skipgram = [loaded_skipgram_model.wv[word] for word in present_words_skip]

# TSNE로 100차원 벡터를 2차원으로 축소
tsne_cbow = TSNE(n_components=2, perplexity=5, random_state=42)
reduced_vectors_cbow = tsne_cbow.fit_transform(np.array(vectors_cbow))
tsne_skip = TSNE(n_components=2, perplexity=5, random_state=42)
reduced_vectors_skipgram = tsne_skip.fit_transform(np.array(vectors_skipgram))

path = '/content/drive/MyDrive/25-2-intelligent-language-processing/' # 시각화 결과를 저장할 경로를 입력하세요.

# 시각화
plt.figure(figsize=(12, 10))
plt.scatter(reduced_vectors_cbow[:, 0], reduced_vectors_cbow[:, 1], edgecolors='black', c='cornflowerblue', s=100, label='CBOW')
for i, word in enumerate(present_words_cbow):
    plt.annotate(word, xy=(reduced_vectors_cbow[i, 0], reduced_vectors_cbow[i, 1]), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')

plt.scatter(reduced_vectors_skipgram[:, 0], reduced_vectors_skipgram[:, 1], edgecolors='black', c='lemonchiffon', s=100, label='Skip-gram')
for i, word in enumerate(present_words_skip):
    plt.annotate(word, xy=(reduced_vectors_skipgram[i, 0], reduced_vectors_skipgram[i, 1]), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')

plt.title('Word Embedding Visualization (CBOW vs. Skip-gram) - text8')
plt.xlabel('TSNE Component 1')
plt.ylabel('TSNE Component 2')
plt.grid(True)
plt.legend()
plt.savefig(f'{path}word_embedding_visualization_text8.png', dpi=150, bbox_inches='tight')
print(f"\n시각화 결과가 '{path}word_embedding_visualization_text8.png' 파일로 저장되었습니다.")
